# Building a Fast and Accurate Dependency Parser
This project implements a neural transition-based **dependency parser**, drawing inspiration from the seminal paper, <a href="https://nlp.stanford.edu/pubs/emnlp2014-depparser.pdf">A Fast and Accurate Dependency Parser using Neural Networks</a>.

Dependency parsing is a fundamental and complex task in Natural Language Processing (NLP), focusing on identifying the grammatical relationships between words in a sentence. Successfully building this system requires sophisticated setup compared to simpler tasks like classification or translation. To manage this complexity, the implementation is broken down into modular functions that allow for incremental development and rigorous testing.

<font color='green'><b>Note:</b> For efficient development and debugging, I recommend leveraging **CPU** until the model is fully prepared for training. The transition to a **GPU** runtime should be reserved for the final training phase to maximize performance.
* Debugging on CPU typically provides clearer error messages and a faster development cycle.
* Using GPU resources efficiently helps maintain access to high-performance computing environments.</font>

# Configuration and Validation Utilities

These initial cells contain essential **helper functions and setup scripts** used to rigorously validate the correctness of the parser implementation as I developed it. These components are strictly for testing and should **not** be modified during the project's implementation phase.

In [1]:
if __name__ == '__main__':
    import cloudpickle as cp
    from urllib.request import urlopen

    testing_bundle = cp.load(urlopen("https://drive.google.com/uc?export=download&id=1-9lWbXuXZYGjJWQRKD7um_83O-I3ER63"))

In [2]:
def sanityCheck(test_fxn, i=None, test_bundle=None, to_print = 'all', do_return=False, display_sent=True, do_raise=True):
    to_test = test_fxn.__name__ + (('_' + str(i)) if i is not None else '')
    if test_fxn.__name__ not in {'get_gold_action', 'get_lc', 'get_rc', 'get_top3_stack_features', 'get_top3_buffer_features'}: assert i is not None
    assert to_test in {'get_gold_action', 'get_lc', 'get_rc', 'get_top3_stack_features', 'get_top3_buffer_features', 'get_lc1_lc2_features_1',
                       'get_lc1_lc2_features_2', 'get_rc1_rc2_features_1', 'get_rc1_rc2_features_2', 'get_llc_rrc_features_1', 'get_llc_rrc_features_2'}
    assert to_print in {'all', 'incorrect', 'none'}
    test_bundle = testing_bundle if test_bundle is None else test_bundle
    sentences, configs, gold_actions, gold_features, _ = test_bundle
    vocab = Vocabulary(sentences)
    testsents = vocab.buildSentences(sentences)
    sents_correct, printag = 0, None # printag will collect the first error you encounter (used in Gradescope)
    for i in range(len(sentences)):

        printsent = "" # printsent is full printout for this sentence (only needed when to_print != 'none')
        sent, gold_deps = testsents[i][0], testsents[i][1]
        if to_test == 'get_gold_action':
            printsent += 'gold_dependencies: ' +str([str(arc) for arc in gold_deps.arcs]) + '\n\n'

        for j in range(len(configs[i])):
            # Manually create Stack, Buffer, Dependencies & ParserConfiguration objects from test case
            stk, buf, deps = configs[i][j]
            stack, buffer, dependencies = Stack([sent[idx] for idx in stk]), Buffer([sent[idx] for idx in buf]), Dependencies(vocab)
            for arc in deps: dependencies.add(sent[arc[0]], sent[arc[1]], arc[2])
            parser_config = ParserConfiguration(sent, vocab)
            parser_config.stack, parser_config.buffer, parser_config.dependencies = stack, buffer, dependencies

            if to_test == 'get_gold_action' or not (gold_actions[i][j] is None or gold_actions[i][j] == 'DONE'): # Don't need to test if Done or None when not testing get_gold_action

                # Call the student code
                arg = stack.get_si(1) if to_test in {'get_lc', 'get_rc'} else (int(to_test[-1]) if to_test[-1] in {'1', '2'} else None)
                if to_test == 'get_gold_action': fxn = lambda: test_fxn(stack, buffer, gold_deps)
                elif to_test in {'get_top3_stack_features', 'get_top3_buffer_features'}: fxn = lambda: test_fxn(parser_config)
                else: fxn = lambda: test_fxn(parser_config, arg)
                tt = to_test[:-2] if to_test[-1] in {'1', '2'} else to_test

                exception=None
                try:
                    yours = fxn()
                except Exception as e:
                    yours = "Raised Exception: " + str(e)
                    exception=e
                correct = gold_actions[i][j] if to_test == 'get_gold_action' else gold_features[i][j][to_test]
                fxn_name = 'get_gold_action(stack, buffer, gold_dependencies)' if to_test == 'get_gold_action'else  tt + ('(parser_config, 1)' if to_test not in to_test in {'get_lc', 'get_rc'} else '(parser_config, ' + str(arg) + ')')

                if to_test in {'get_lc', 'get_rc'} and exception is None:
                    if type(yours) != list or (len(yours) > 0 and type(yours[0]) != Arc):
                        yours = 'Your ' + to_test + '(...) did not return a list of Arcs' # note: exact quote used below, so if need to change, change in both places
                    else: yours = [str(arc) for arc in yours]
                # if random.random() < 0.05: yours = [None, None, None, None, None, None] # simulate getting it wrong
                is_correct = yours == correct

                # Make the printouts!
                printsent += 'Step ' + str(j+1) + ' | stack: ' + str([str(word) if to_test != 'get_gold_action' else word.word for word in stack]) + '\tbuffer: ' + str([str(word) if to_test != 'get_gold_action' else word.word for word in buffer]) + (('\tdependencies: ' + str([str(arc) for arc in dependencies.arcs])) if to_test != 'get_gold_action' else "") + '\n'
                printsent += '\tTesting '+ fxn_name +':\n'
                printsent += '\t\tYour Result: ' + str(yours) + '\t\tCorrect Result: ' + str(correct) + ('\t\tCorrect!' if is_correct else "") + '\n'
                if not is_correct:
                    printsent += '\t\tIncorrect! Halting parse of this sentence.\n\n'
                    if printag is None:
                        statement=yours if yours == 'Your ' + to_test + '(...) did not return a list of Arcs' else str(yours)[:20]+("... " if len(str(yours)) > 20 else "")
                        statement = "Your first error (on a hidden test sentence): You returned " + statement + "; expected " + str(correct) + "."
                        printag = statement if exception is None else (statement, exception)
                    if to_print != 'none':
                        print("Testing Sentence " + str(i+1) + '...')
                        if display_sent: display_sentence(sentences[i])
                        print(printsent)
                        print('Test Sentence ' + str(i+1) + ' failed.')
                        print(('\n' + '-'*100 + '\n'))
                    if do_raise and exception is not None:
                        raise exception
                    break
                else: printsent += '\n'

        else:
            sents_correct += 1
            if to_print == 'all':
                print("Testing Sentence " + str(i+1) + '...')
                if display_sent: display_sentence(sentences[i])
                print(printsent)
                print('Test Sentence ' + str(i+1) + ' passed!')
                print(('\n' + '-'*100 + '\n'))
    score = sents_correct / len(sentences)
    print(sents_correct, '/', len(sentences), '=', str(score*100)+'%', 'of test cases passed!')
    if do_return:
        return score, printag

def sanityCheck_generate_training_examples(test_fxn, feat_extract = None, test_bundle=None, to_print='all', do_return=False, display_sent=True, do_raise=True):
    assert test_fxn.__name__ == 'generate_training_examples' and (feat_extract is None or feat_extract.__name__ == 'extract_features')
    test_bundle = testing_bundle if test_bundle is None else test_bundle
    sentences, _, _, _, gold_examples = test_bundle
    vocab = Vocabulary(sentences)
    testsents = vocab.buildSentences(sentences)
    sents_correct, printag = 0, None
    for i in range(len(sentences)):
        sent, gold_deps = testsents[i][0], testsents[i][1]
        exception=None
        try:
            your_examples = test_fxn(sent, gold_deps, vocab, feat_extract=feat_extract if feat_extract is not None else lambda p: [])
        except Exception as e:
            your_examples = "Raised Exception: " + str(e)
            exception=e
        # if random.random() < 0.5: your_examples = [None, None, None, None, None, None, None, None] # simulate getting it wrong
        is_correct = your_examples == gold_examples[i][0 if feat_extract == None else 1]
        if to_print == 'all' or to_print == 'incorrect' and not is_correct:
            print("Testing Sentence " + str(i+1) + '...')
            if display_sent: display_sentence(sentences[i])
            print("sentence: " + str([word.word for word in sent]) + '\tgold_dependencies: ' + str([str(arc) for arc in gold_deps.arcs]))
            print("\tTesting generate_training_examples(sentence, gold_dependencies, vocab" + ('' if feat_extract is None else ', feat_extract=extract_features') + "):")
            print('\t\tYour Training Examples:   ', your_examples, '\n\t\tCorrect Training Examples:', gold_examples[i][0 if feat_extract == None else 1])
            print('\t\tCorrect!\n\n' if is_correct else '\t\tIncorrect!\n\n')
        if not is_correct and printag is None:
            statement = "Your first error (on a hidden test sentence): You returned " + str(your_examples)[0:20]+("... " if len(str(your_examples)) > 20 else "") + "; expected " + str(gold_examples[i]) + "."
            printag = statement if exception is None else (statement, exception)
        if to_print == 'all' or to_print == 'incorrect' and not is_correct:
            print(('Test Sentence ' + str(i+1) + ' passed!') if is_correct else ('Test Sentence ' + str(i+1) + ' failed.'))
            print('\n'+ '-'*100, '\n')
        if is_correct: sents_correct += 1
        if do_raise and exception is not None:
            raise exception

    score = sents_correct / len(sentences)
    print(sents_correct, '/', len(sentences), '=', str(score*100)+'%', 'of test cases passed!')
    if do_return:
        return score, printag

def sanityCheckStackBuffer():
    sanity_stack = Stack()
    item1 = Word('she', 'PRP', 0)
    item2 = Word('is', 'VBP', 1)
    item3 = Word('working', 'VBG', 2)
    sanity_stack.push(item1)
    sanity_stack.push(item2)
    sanity_stack.push(item3)
    p1 = False
    p2 = False
    p3 = False
    pop_item = sanity_stack.pop()
    if (pop_item.idx == 2 and pop_item.word == 'working' and pop_item.pos == 'VBG'):
        p1 = True
    pop_item = sanity_stack.pop()
    if (pop_item.idx == 1 and pop_item.word == 'is' and pop_item.pos == 'VBP'):
        p2 = True
    pop_item = sanity_stack.pop()
    if (pop_item.idx == 0 and pop_item.word == 'she' and pop_item.pos == 'PRP'):
        p3 = True
    if p1 and p2 and p3:
        print("Stack sanity check PASSED")
    else:
        print("Stack sanity check FAILED")

    sentence = [item1, item2, item3]
    sanity_buffer = Buffer(sentence)
    p1 = False
    p2 = False
    p3 = False
    pop_item = sanity_buffer.pop()
    if (pop_item.idx == 0 and pop_item.word == 'she' and pop_item.pos == 'PRP'):
        p1 = True
    pop_item = sanity_buffer.pop()
    if (pop_item.idx == 1 and pop_item.word == 'is' and pop_item.pos == 'VBP'):
        p2 = True
    pop_item = sanity_buffer.pop()
    if (pop_item.idx == 2 and pop_item.word == 'working' and pop_item.pos == 'VBG'):
        p3 = True
    if p1 and p2 and p3:
        print("Buffer sanity check PASSED")
    else:
        print("Buffer sanity check FAILED")

count_parameters = lambda model: sum(p.numel() for p in model.parameters() if p.requires_grad)

def sanityCheckModel(all_test_params, model, expected_outputs, init_or_forward, batch_sizes=None):
    print('--- TEST: ' + ('Number of Model Parameters (tests __init__(...))' if init_or_forward=='init' else 'Output shape of forward(...)') + ' ---')

    for tp_idx, (test_params, expected_output) in enumerate(zip(all_test_params, expected_outputs)):
        if init_or_forward == "forward":
            assert len(batch_sizes) == len(all_test_params)
            texts = torch.randint(1,4,(batch_sizes[tp_idx], test_params['n_features']))

        # Construct the student model
        tps = {k:v for k, v in test_params.items() if k != 'batch_size'}
        stu_parser_model = model(**tps)

        if init_or_forward == "forward":
            with torch.no_grad():
                stu_out = stu_parser_model(texts)
            ref_out_shape = expected_output

            has_passed = torch.is_tensor(stu_out)
            if not has_passed: msg = 'Output must be a torch.Tensor; received ' + str(type(stu_out))
            else:
                has_passed = stu_out.shape == ref_out_shape
                msg = 'Your Output Shape: ' + str(stu_out.shape)


            status = 'PASSED' if has_passed else 'FAILED'
            message = '\t' + status + "\t Init Input: " + str({k:v for k,v in tps.items()}) + '\tForward Input Shape: ' + str(texts.shape) + '\tExpected Output Shape: ' + str(ref_out_shape) + '\t' + msg
            print(message)
        else:
            assert batch_sizes is None
            stu_num_params = count_parameters(stu_parser_model)
            ref_num_params = expected_output
            comparison_result = (stu_num_params == ref_num_params)

            status = 'PASSED' if comparison_result else 'FAILED'
            message = '\t' + status + "\tInput: " + str({k:v for k,v in test_params.items()}) + ('\tExpected Num. Params: ' + str(ref_num_params) + '\tYour Num. Params: '+ str(stu_num_params))
            print(message)

        del stu_parser_model

# Data Processing and Initialization

The first crucial step in building the parser involves preparing the raw data for consumption by the neural network model. This includes loading the dependency treebanks, standardizing the format, and initializing the necessary data structures and lookup tables.

In [3]:
import numpy as np
from spacy import displacy
import random

## Reading and Visualizing the Dependency Treebank

To train the parser effectively, I utilized a standard, preprocessed corpus. For context, the data originates from a recognized source in NLP:
* Specifically, the dataset is based on the **UD_English** corpus (version 1.4) from the Universal Dependencies project.
* The structure and concepts behind this framework can be explored further at https://universaldependencies.org/.
* Information regarding the data's usage license is available here: https://lindat.mff.cuni.cz/repository/xmlui/page/licence-UD-1.4

The following utility scripts load the prepared data and report the total number of sentences available for training and testing. These data loading functions are standard and **do not require modification**.

In [4]:
def load_data():
    train_set = cp.load(urlopen("https://drive.google.com/uc?export=download&id=1N4B4bC4ua0bFMIYeNdtNQQ72LxxKLPas"))
    test_set = cp.load(urlopen("https://drive.google.com/uc?export=download&id=1TE2AflhABbz41dLMGmD1kTm7WevYpU31"))
    return train_set, test_set

In [5]:
if __name__ == '__main__':
    train_set, test_set = load_data()
    print("Num. Train Examples:", len(train_set))
    print("Num. Test Examples: ", len(test_set))

Num. Train Examples: 12543
Num. Test Examples:  2002


## Analyzing the Training Data Structure

The training corpus consists of sentences annotated with **labeled dependency trees**. This is crucial, as the primary objective of this project is to build a model capable of accurately predicting both the **dependency arcs** (the connections between words) and their associated **labels** (the grammatical relationship type) for any new, unseen sentence. Visualizing this data helps in understanding the target output structure.

In [6]:
def display_sentence(sent):
    res = {'words': [{'text': "<ROOT>", 'tag': 'POS_ROOT'}], 'arcs': []}
    for i in range (len(sent['word'])):
        res['words'].append({'text': sent['word'][i], 'tag': sent['pos'][i]})
        s = i + 1
        e = sent['head'][i]
        direc = "left"
        if s > e:
            s = sent['head'][i]
            e = i + 1
            direc = 'right'
        cur = {'start': s, 'end': e, 'label': sent['label'][i], 'dir': direc}
        res['arcs'].append(cur)
    displacy.render(res, style="dep", manual=True, jupyter=True, options={'distance': 70})


In [7]:
if __name__ == '__main__':
    MIN_SENT_LEN, MAX_SENT_LEN = 4, 17 # You can change this if you're interested in seeing shorter/longer sentences
    for x in random.sample(list(filter(lambda x: len(x['word']) >= MIN_SENT_LEN and len(x['word']) <= MAX_SENT_LEN, train_set)), 5):
        display_sentence(x)

## Initializing the Model's Lookup Tables

A critical component of this neural dependency parser is the **`Vocabulary` class**. This class serves as the core mapping system, translating linguistic elements into the numerical indices required by the neural network's embedding layers.

Specifically, the `Vocabulary` is responsible for:

* **Mapping Words, POS Tags, and Labels:** Assigning a unique integer ID to every encountered **word**, **part-of-speech tag (POS)**, and **dependency label**. This numerical representation is essential for creating the dense feature vectors (embeddings).
* **Enumerating Transitions:** Defining the complete set of possible actions the parser can take. If the corpus contains $n$ unique dependency labels, the parser's action space is defined by **$2n+1$ transitions**:
    * **Left-Arc (LA):** An action for each of the $n$ labels.
    * **Right-Arc (RA):** An action for each of the $n$ labels.
    * **Shift (S):** A single action for moving a word from the buffer to the stack.

This comprehensive set of actions is what the neural network ultimately predicts during parsing. This setup code is complete and **does not require modification**.

In [8]:
class Vocabulary(object):
    def __init__(self, dataset):

        UNK = '<UNK>'
        NULL = '<NULL>'
        ROOT = '<ROOT>'

        # Find the label of the root
        root_labels = list([l for ex in dataset for (h, l) in zip(ex['head'], ex['label']) if h == 0])
        assert len(set(root_labels)) == 1
        self.root_label = root_labels[0]

        # Create mapping from transitions to ids
        labels = sorted(list(set([w for ex in dataset for w in ex['label'] if w != self.root_label]))) # list of unique non-root labels
        labels = [self.root_label] + labels # add root label too
        self.n_labels = len(labels)

        transitions = ['LA-' + l for l in labels] + ['RA-' + l for l in labels] + ['S']
        self.n_trans = len(transitions) # 2*n_labels + 1
        self.tran2id = {t: i for (i, t) in enumerate(transitions)}
        self.id2tran = {i: t for (i, t) in enumerate(transitions)}

        # Create mapping from word, pos, & label to id
        # Do labels first
        self.LABEL_PREFIX = '<l>:'
        self.tok2id = {self.LABEL_PREFIX + l: i for (i, l) in enumerate(labels)}
        self.LABEL_NULL = self.tok2id[self.LABEL_PREFIX + NULL] = len(self.tok2id) # Add null label in

        # Do pos's
        self.POS_PREFIX = '<p>:'
        all_pos = sorted(set([self.POS_PREFIX + w for ex in dataset for w in ex['pos']])) # Get pos's
        self.tok2id.update({w: index + len(self.tok2id) for (index, w) in enumerate(all_pos)}) # Add poses in
        self.POS_NULL = self.tok2id[self.POS_PREFIX + NULL] = len(self.tok2id) # Add null pos
        self.POS_ROOT = self.tok2id[self.POS_PREFIX + ROOT] = len(self.tok2id) # Add root pos
        self.n_pos = 2 + len(all_pos) # +3 for null, root

        # Do words
        all_word = sorted(set([w for ex in dataset for w in ex['word']]))
        self.tok2id.update({w: index + len(self.tok2id) for (index, w) in enumerate(all_word)}) # Add words in
        self.WORD_UNK = self.tok2id[UNK] = len(self.tok2id) # Add unk word
        self.WORD_NULL = self.tok2id[NULL] = len(self.tok2id) # Add null word
        self.WORD_ROOT = self.tok2id[ROOT] = len(self.tok2id) # Add root word
        self.n_words = 3 + len(all_word) # +3 for unk, null, root

        self.id2tok = {v: k for (k, v) in self.tok2id.items()} # Flip it
        self.n_tokens = len(self.tok2id)

    def printStats(self):
        print('Num. labels:', self.n_labels)
        print('Num. transitions (2*n_labels + 1):', self.n_trans)
        print('Num. pos:', self.n_pos)
        print('Num. words:', self.n_words)
        print('Num. tokens:', self.n_tokens)


    def buildSentences(self, examples):
        processed_sentences = []
        for ex in examples:
            # Initialize words & dependencies
            words = [Word('<ROOT>', '<POS_ROOT>', 0, self.WORD_ROOT, self.POS_ROOT)]
            deps = []

            # Loop over words in sentence
            for i  in  range(len(ex['word'])):
                w = ex['word'][i]
                word_id = (self.tok2id[w] if w in self.tok2id else self.WORD_UNK)
                pos = self.POS_PREFIX + ex['pos'][i]
                pos_id = self.tok2id[pos]
                word = Word(ex['word'][i], ex['pos'][i],i+1, word_id, pos_id)

                words.append(word)
                deps.append((ex['head'][i], word, ex['label'][i] ))

            # Create dependencies
            dependencies = Dependencies(self)
            for dep in deps:
                dependencies.add(words[dep[0]], dep[1], dep[2])

            processed_sentences.append((words, dependencies))

        return processed_sentences

## Analyzing Vocabulary Statistics

After defining the `Vocabulary` structure, the next logical step is to examine the scale of the linguistic data the model will handle. **Running this script reports key statistics** derived from the corpus, providing necessary context on the number of unique elements (words, POS tags, labels) and the total size of the transition space. This introspection is vital for configuring the embedding layer dimensions and overall model capacity.

In [9]:
if __name__ == '__main__':
    Vocabulary(train_set).printStats()

Num. labels: 47
Num. transitions (2*n_labels + 1): 95
Num. pos: 52
Num. words: 16635
Num. tokens: 16735


# Core Data Structure Implementation

The operational engine of the transition-based parser relies on precisely defined data structures to manage the sentence processing state. In this phase of the project, I implemented the fundamental components that collectively define the parser's configuration:

* **The `Stack`:** This structure holds words that are currently being examined for potential head/dependent relationships.
* **The `Buffer`:** This structure holds the remaining, unprocessed words of the input sentence.
* **The `ParserConfiguration`:** This class encapsulates the current state of the stack, buffer, and the set of dependency arcs established so far.

I also developed the essential logic to **update these data structures based on the chosen transition** (Shift, Left-Arc, or Right-Arc), ensuring the parser moves correctly from one state to the next.

## Defining Foundational Data Structures

To manage the parser's state effectively, I first established several **key data classes**. These structures are used extensively throughout the project, and a clear understanding of the data they encapsulate and the methods they provide is essential for successful implementation. They represent the core components that track the state of the sentence being processed. These initial definitions are complete and **do not require modification**.

In [10]:
class Word(object):
    '''
    Represents a word in the sentence.
    '''

    def __init__(self, word, pos, idx, word_id=None, pos_id=None):
        self.word = word
        self.pos = pos
        self.idx = idx
        self.word_id = word_id
        self.pos_id = pos_id

    def __str__(self):
        return 'Word(idx=' + str(self.idx) + ", word='" + self.word+"', pos='"+self.pos+"', word_id="+str(self.word_id)+', pos_id='+ str(self.pos_id) +')'

    def copy(self):
        return Word(self.word, self.pos, self.idx, self.word_id, self.pos_id)

    def __eq__(self, obj):
        if not isinstance(obj, Word): return False
        if obj.idx == self.idx:
            assert obj.word == self.word and obj.pos == self.pos and obj.word_id == self.word_id and obj.pos_id == self.pos_id, 'Your Word object has been corrupted.'
        return obj.idx == self.idx


class Arc(object):
    '''
    Represents an arc between two words.
    '''

    def __init__(self, head, dependent, label, label_id):
        self.head=head # Word object
        self.dependent=dependent # Word object
        self.label=label
        self.label_id = label_id

    def __str__(self):
        return 'Arc(head_idx='+str(self.head.idx)+', dep_idx='+str(self.dependent.idx)+', label_id='+ str(self.label_id)+')'


class Dependencies(object):
    '''
    Represents the dependency arcs in a sentence.
    '''

    def __init__(self, vocab):
        self.arcs = []
        self.vocab = vocab
        self.dep_to_head_mapping = {} # For fast lookup

    def add(self, head, dependent, label):
        '''
        Add a dependency from head to dependent with label.
        Inputs:
            head: Word object
            dependent: Word object
            label: str
        '''
        assert label[:3] != 'LA-' and label[:3] != 'RA-', 'You need to pass in just the label to add(...), not the entire action.'
        assert head is not None and dependent is not None, "You must pass in two Word objects to add(...)."
        self.arcs.append(Arc(head, dependent, label, self.vocab.tok2id[self.vocab.LABEL_PREFIX+label]))
        assert dependent.idx not in self.dep_to_head_mapping
        self.dep_to_head_mapping[dependent.idx] = self.arcs[-1]

    def getArcToHead(self, dependent):
        '''
        Returns the Arc that connects the head of dependent to dependent.
        Inputs:
            dependent: Word object
        '''
        if dependent.idx == 0: # Special case for ROOT
            return Arc(None, dependent, None, None)
        return self.dep_to_head_mapping[dependent.idx]

    def __iter__(self):
        return iter(self.arcs) # Allows you to iterate "for x in Dependencies(...)"

## Implementing Core Stack and Buffer Operations

To enable the parser's transition mechanisms, I needed to define the fundamental state-manipulation logic for both the Stack and the Buffer structures. This involved **implementing the essential `push(...)` and `pop(...)` methods for the `Stack`**—allowing elements to be added or removed from the top—and **implementing the `pop(...)` method for the `Buffer`**, which sequentially removes the next word for processing. These operations are crucial for executing the Shift and Arc transitions.

In [11]:
class Stack(object):
    def __init__(self, input=[]):
        '''
        Initialize an (empty) stack.
        '''
        self.stack = [word.copy() for word in input]

    def push(self, item):
        '''
        Push item onto (the end of) self.stack. Returns nothing.
        '''
        self.stack.append(item)

    def pop(self):
        '''
        Pop item from (the end of) self.stack. Returns the item popped.
        '''
        assert len(self.stack) > 0
        return self.stack.pop()

    def get_si(self, i):
        '''
        Returns si (the ith element of the stack) if it exists, otherwise None.
        '''
        assert i > 0, 'Must provide i > 0'
        return self.stack[-i] if len(self.stack) >= i else None

    def __getitem__(self, idx):
        return self.stack[idx]

    def __len__(self):
        return len(self.stack)

    def __str__(self):
        return str([str(x) for x in self.stack])


class Buffer(object):
    def __init__(self, sentence):
        '''
        Initialize as a list of words in sentence.
        '''
        self.buffer = [word.copy() for word in sentence]

    def pop(self):
        '''
        Pop item from (the beginning of) self.buffer. Returns the item popped.
        '''
        assert len(self.buffer) > 0
        return self.buffer.pop(0)

    def get_bi(self, i):
        '''
        Returns bi (the ith element of the buffer) if it exists, otherwise None.
        '''
        assert i > 0, 'Must provide i > 0'
        return self.buffer[i-1] if len(self.buffer) >= i else None

    def __getitem__(self, idx):
        return self.buffer[idx]

    def __len__(self):
        return len(self.buffer)

    def __str__(self):
        return str([str(x) for x in self.buffer])

## Self-Validation of Stack and Buffer Functionality

To ensure the core data structures are implemented correctly, I included a **sanity check script**. This script performs essential tests on the `Stack` and `Buffer` classes, simulating typical operations. While this self-validation confirms basic functionality, it is designed primarily as a debugging aid and does **not** replace the comprehensive, hidden tests necessary for full deployment confidence. Passing this check verifies the fundamental mechanisms of element manipulation.

In [12]:
if __name__ == '__main__':
    sanityCheckStackBuffer()

Stack sanity check PASSED
Buffer sanity check PASSED


## Defining and Implementing the Parser's Core State Transitions

The next crucial stage involves integrating the `Stack`, `Buffer`, and `Dependencies` into a single **`ParserConfiguration`** class. This class represents the complete state of the sentence processing at any given moment.

My primary task in this implementation was developing the **`parse_step(self, transition)`** method. This function is the engine that drives the parser, responsible for correctly modifying the configuration's internal structures based on the transition chosen by the neural network classifier.

The implementation handles the three fundamental transition types as follows:

* **Shift (`transition = 'S'`):** This operation moves the first word from the buffer ($\beta$) and places it onto the stack ($\sigma$).
    $$\left(\sigma, w_k|\beta, A\right) \Rightarrow \left(\sigma|w_k, \beta, A\right)$$
* **Left-Arc (`transition = 'LA-label'`):** This transition establishes a dependency arc, where the second word on the stack ($w_i$) becomes the dependent of the first word on the stack ($w_j$). $w_i$ is then removed, and the new arc is recorded in $A$.
    $$\left(\sigma |w_i w_j , \beta, A\right) \Rightarrow \left(\sigma |w_j, \beta, A \cup \{(w_j, \text{label}, w_i)\}\right)$$
* **Right-Arc (`transition = 'RA-label'`):** This transition establishes a dependency arc, where the first word on the stack ($w_j$) becomes the dependent of the second word on the stack ($w_i$). $w_j$ is then removed, and the new arc is recorded in $A$.
    $$\left(\sigma |w_i w_j , \beta, A\right) \Rightarrow \left(\sigma |w_i, \beta, A \cup \{(w_i, \text{label}, w_j)\}\right)$$

This logic relies directly on the previously implemented `push(...)` and `pop(...)` methods for the Stack and Buffer, along with the `add_arc(...)` functionality provided by the `Dependencies` structure.

In [13]:
class ParserConfiguration(object):
    def __init__(self, sentence, vocab):
        '''
        Inputs:
            sentence: list of Word objects
            vocab: Vocabulary object
        '''

        self.vocab = vocab

        assert sentence[0].word_id == self.vocab.WORD_ROOT
        self.stack = Stack([sentence[0]]) # Initialize stack with ROOT
        self.buffer = Buffer(sentence[1:]) # Initialize buffer with sentence
        self.dependencies = Dependencies(vocab)

    def parse_step(self, transition):
        '''
        Update stack, buffer, and dependencies based on transition.
        Inputs:
            transition: str, "S", "LA-label", or "RA-label", where label is a valid label
        '''
        assert transition in self.vocab.tran2id

        if transition == "S":
            self.stack.push(self.buffer.pop())
        elif transition[:3] == "RA-":
            dependent = self.stack.pop()
            head = self.stack.get_si(1)
            label = transition[3:]
            self.dependencies.add(head, dependent, label)
        elif transition[:3] == "LA-":
            head = self.stack.pop()
            dependent = self.stack.pop()
            label = transition[3:]
            self.dependencies.add(head, dependent, label)
            self.stack.push(head)

# Generating Training Examples from Gold Parses

The transition-based parsing approach requires a supervised learning signal at every step. Therefore, the core of this phase is to **transform the static gold dependency parses into a dynamic sequence of training examples**. For each sentence, I need to generate a series of configurations, where the input features describe the current state of the parser, and the target label is the *optimal* transition that moves the parser closer to the final gold structure.

This process involves two main components:

1.  **Oracle Implementation:** Developing an algorithm (the "oracle") that determines the **correct next parser action** at any given time step, based on the final gold dependency tree.
2.  **Feature Extraction:** Implementing the logic to accurately **extract the relevant features** from the current parser configuration (Stack, Buffer, and existing dependencies), which will be fed into the neural network classifier.

## Defining the Oracle: Determining the Gold Transition

To generate accurate training examples, I developed the **`get_gold_action(stack, buffer, gold_dependencies)`** function, which acts as the **parsing oracle**. Given the current state of the parser (the `stack` and `buffer`) and the final correct dependency structure (`gold_dependencies`), this function determines the single optimal action (Shift, Left-Arc, Right-Arc, or termination) that should be executed next.

The logic for selecting the gold action is implemented based on the following precedence rules. Let $s_i$ be the $i$-th element of the stack (where $s_1$ is the top element) and $h(w)$ be the gold head of word $w$.

### Oracle Selection Logic

1.  **Termination and Initialization Check:**
    * If the stack only contains the `ROOT` token:
        * If the buffer is not empty, the action must be **Shift (`S`)** to begin processing the sentence.
        * If the buffer is empty, the parse is successfully complete: **`DONE`**.

2.  **Left-Arc Check (Prioritized):**
    * If the head of the second word on the stack ($s_2$) is the top word ($s_1$), the correct action is **Left-Arc** (`LA-label`). $s_2$ is a dependent of $s_1$ and can be removed.
        * $h(s_2) = s_1 \Rightarrow \text{Return } \mathbf{LA-label}$ (where $\text{label}$ is the gold arc type).

3.  **Right-Arc Check (Requires Safety Condition):**
    * If the head of the top word on the stack ($s_1$) is the second word ($s_2$), **AND** $s_1$ has no remaining dependents in the buffer ($\beta$):
        * $h(s_1) = s_2 \text{ and } \forall b_i \in \beta, h(b_i) \neq s_1 \Rightarrow \text{Return } \mathbf{RA-label}$ (where $\text{label}$ is the gold arc type).
    * *This secondary condition (no buffer word depends on $s_1$) is essential for the arc-standard system to maintain the property of **projectivity** and ensure all dependents are attached before $s_1$ is reduced.*

4.  **Default Action:**
    * Otherwise:
        * If the buffer is not empty, the default action is **Shift (`S`)** to continue processing.
        * If the buffer is empty, the parse is considered non-projective or failed, and the function returns **`None`**.

### Implementation Details

I utilize the provided utility methods: `stack.get_si(i)` and `buffer.get_bi(i)` to access words, and `gold_dependencies.getArcToHead(w)` to retrieve the gold head and label for a word.

In [14]:
def get_gold_action(stack, buffer, gold_dependencies):
    '''
    Given stack & buffer, compute the next gold action based on gold_dependencies.
    Args:
        - stack: Stack object
        - buffer: Buffer object
        - gold_dependencies: Dependencies object
    Returns:
        - action: str; 'S', 'LA-label', or 'RA-label', where 'label' is a valid label. Return None if no action possible and 'DONE' if the parse is complete.
    '''

    if len(stack) == 1:
        if len(buffer) == 0:
            return "DONE"
        else:
            return "S"
            
    s1 = stack.get_si(1)
    s2 = stack.get_si(2)
    
    s1_arc = gold_dependencies.getArcToHead(s1)
    s2_arc = gold_dependencies.getArcToHead(s2)

    if s2_arc.head == s1:
        return f"LA-{s2_arc.label}"
    
    if s1_arc.head == s2:
        s1_is_needed_in_buffer = False
        
        for word_in_buffer in buffer.buffer: 
            buffer_word_head = gold_dependencies.getArcToHead(word_in_buffer).head
            
            if buffer_word_head == s1:
                s1_is_needed_in_buffer = True
                break
        
        if not s1_is_needed_in_buffer:
            return f"RA-{s1_arc.label}"
            
    if len(buffer) != 0:
        return "S"
    else:
        return None

## Validating Gold Action Selection (Sanity Check)

To rigorously test the implemented parsing oracle (`get_gold_action`), I run it against a comprehensive validation set of **10 sentences**. This set is structured to cover various scenarios encountered during parsing:

* **Sentence 1:** The primary example used for instructional reference, ensuring the core parsing flow is correct.
* **Sentences 2–9:** Eight specialized, **artificial sentences** designed specifically to test edge cases and boundary conditions of the Shift, Left-Arc, and Right-Arc transitions.
* **Sentence 10:** A real example extracted directly from the training corpus to ensure real-world data compatibility.

### Debugging and Reporting Controls

This validation script includes several utilities to aid in debugging:

* **Pre-defined States:** For each test sentence, the corresponding **correct stack and buffer configurations** and the **expected gold action** are hard-coded, allowing for precise comparison against the function's output.
* **`to_print` Argument:** Controls the output verbosity:
    * `'all'`: Shows the result for every single test sentence.
    * `'incorrect'`: Only prints details for sentences where the function returns the wrong action.
    * `'none'`: Suppresses all sentence-level output, only reporting the final score.
* **`do_raise` Toggle:** Used to control error handling. If set, exceptions immediately halt execution; if unset, errors are suppressed, allowing the test to run to completion and report the overall success rate.

This self-contained testing suite is essential for ensuring the oracle's reliability before moving to the feature extraction and neural network training phases.

In [15]:
if __name__ == '__main__':
    sanityCheck(get_gold_action, to_print='incorrect', do_raise=True)

10 / 10 = 100.0% of test cases passed!


## Orchestrating the Training Data Generation Pipeline

The goal of this function is to automate the crucial process of converting static, gold-standard sentences into the dynamic sequence of input-output pairs needed to train the neural network classifier. **Each sentence must be transformed into a set of distinct training examples**, where:

1.  **Input:** The current partial **parser configuration** (Stack, Buffer, Dependencies).
2.  **Output (Label):** The **gold action** determined by the oracle for that configuration.

I implemented the core loop that manages this transformation:

* It initializes a parser configuration for a given sentence.
* It repeatedly calls the **oracle function** (implemented previously) to determine the next correct action.
* It uses the `parse_step` method to apply that action, advancing the configuration.
* At each step, it captures the current configuration and the gold action, forming a training example.

Crucially, for now, the configuration's feature representation is handled by a placeholder function, **`feat_extract(parser_config)`**, which will be fully developed later. This modular approach allows the logic of the training example pipeline to be validated independently of the final feature extraction details.

In [16]:
def generate_training_examples(sentence, gold_dependencies, vocab, feat_extract = lambda parser_config: []):
    '''
    Create training instances for sentence.
    Inputs:
        sentence: list of Word objects
        gold_dependencies: Dependencies object that contains the complete gold dependency tree
        vocab: Vocabulary object
        feat_extract: Feature extraction function
    Outputs:
        training_examples: List of tuples (features, label), where features is a list and label is a string
    Pseudocode:
        (1) Initialize your parser configuration (note that the __init__ method of ParserConfiguration creates the stack & buffer for you)
        (2) Repeatedly call get_gold_action(...) on your current parser confuration until the gold action is 'DONE'
        (3) If the gold action is None at any step, return []  (indicating the sentence cannot be parsed; it is non-projective)
        (4) Otherwise, append tuple (features, gold action) to training_examples, where features is the result of calling feat_extract on your parser configuration
        (5) Update your parser configuration according to the gold action
        (6) Return training_examples
    '''

    training_examples = []

    parser_config = ParserConfiguration(sentence, vocab)

    while True:
        gold_action = get_gold_action(
            parser_config.stack, 
            parser_config.buffer, 
            gold_dependencies
        )
        
        if gold_action is None:
            return []
        
        if gold_action == 'DONE':
            break

        features = feat_extract(parser_config)
        training_examples.append((features, gold_action))
        parser_config.parse_step(gold_action)
    
    
    return training_examples

## Validating the Training Example Generation Pipeline

To verify the correctness of the **training example generation function**, I run a comprehensive sanity check using the same **10 structured test sentences** previously used for oracle validation.

This test confirms that the function successfully iterates through a complete parse for each sentence, correctly invoking the oracle at every step and recording the resulting configuration-action pair. By validating that the pipeline terminates successfully and yields the expected number of steps for these known cases, I ensure the structural integrity of the training data generated for the neural network.

In [17]:
if __name__ == '__main__':
    sanityCheck_generate_training_examples(generate_training_examples, to_print='incorrect', do_raise=True)

10 / 10 = 100.0% of test cases passed!


## Creating the Full Training Dataset

This script executes the core data pipeline by calling the **`generate_training_examples(...)` function** on every sentence within the complete dataset. This process aggregates all the configuration-action pairs into a massive collection, which forms the final, comprehensive training dataset required to train the neural network classifier. This function is a utility for data aggregation and **does not require modification**.

In [18]:
def generate_all_training_examples(vocab, sentences, feat_extract = lambda parser_config: []):
    '''
    Generate training examples for all sentences.
    '''
    all_training_examples = []
    successful_sents = 0
    for sentence in sentences:
        training_examples = generate_training_examples(sentence[0], sentence[1], vocab, feat_extract)
        if training_examples != []:
            all_training_examples += training_examples
            successful_sents += 1

    print("Successfully generated training examples for", successful_sents, "/", len(sentences), "sentences")
    print("Number of training examples:", len(all_training_examples))

    return all_training_examples

In [19]:
if __name__ == '__main__':
    _vocab = Vocabulary(train_set) # Variable just used in this cell
    generate_all_training_examples(_vocab, _vocab.buildSentences(train_set))

Successfully generated training examples for 11888 / 12543 sentences
Number of training examples: 373100


## Feature Engineering: Extracting Input Data for the Neural Network

The final step in preparing the training data is implementing the **feature extraction logic**. The neural network relies on a compact, dense representation of the parser's state to predict the next optimal action. Following the methodology outlined in the paper (Section 3.1), I implemented a system to extract **48 relevant features** from every parser configuration: 18 word features, 18 POS tag features, and 12 dependency label features.

A "feature" in this context is simply the **ID (index)** of a word, POS tag, or dependency label within the global `Vocabulary`. These IDs will be mapped to dense embedding vectors during the neural network training phase.

### Feature Extraction Plan (Total 48 Features)

I will systematically implement five functions, each responsible for extracting a specific group of features from the configuration, returning separate lists for the word IDs, POS IDs, and Label IDs.

1.  **Top Stack and Buffer Elements (18 Word / 18 POS Features):**
    * Word and POS features for the top three items on the stack: **$s_1, s_2, s_3$**.
    * Word and POS features for the top three items in the buffer: **$b_1, b_2, b_3$**.

2.  **First and Second Left/Right Children (8 Word / 8 POS / 8 Label Features):**
    * Word, POS, and Label features for the immediate children of the top two stack items: **$lc_1(s_1), lc_2(s_1), lc_1(s_2), lc_2(s_2)$**.
    * Word, POS, and Label features for the immediate right children of the top two stack items: **$rc_1(s_1), rc_2(s_1), rc_1(s_2), rc_2(s_2)$**.

3.  **Grandchildren (4 Word / 4 POS / 4 Label Features):**
    * Word, POS, and Label features for the "grandchildren" of the top two stack items: **$lc_1(lc_1(s_1)), rc_1(rc_1(s_1)), lc_1(lc_1(s_2)), rc_1(rc_1(s_2))$**.

**Note on Non-Existent Elements:** If any required element (e.g., $s_3$ when the stack is small, or a child that doesn't exist) cannot be found, the feature ID should default to the special `<NULL>` token ID provided by the `Vocabulary`.

## Feature Implementation: Top Stack and Buffer Elements

The initial set of features focuses on the most immediate context available to the parser: the words on the stack waiting to be processed and the words coming up next in the buffer.

I implemented two key functions to capture this context, ensuring a fixed-size feature vector by using the `<NULL>` token ID when an element does not exist (e.g., a stack with fewer than three items).

* **`get_top3_stack_features(parser_config)`:** This function **extracts the word and POS IDs** for the three elements at the top of the stack ($s_1, s_2, s_3$).
* **`get_top3_buffer_features(parser_config)`:** This function **extracts the word and POS IDs** for the three elements at the front of the buffer ($b_1, b_2, b_3$).

These functions are critical for creating the initial 18 word and 18 POS features used as input to the neural network.

In [20]:
def get_top3_stack_features(parser_config):
    '''
    Get the word and POS features for s1, s2, and s3 (the top 3 items on the stack)
    Returns:
        word_features: List of word ids for s1, s2, s3 (use vocab.WORD_NULL if a word does not exist)
        pos_features: List of POS ids for s1, s2, s3 (use vocab.POS_NULL if a word does not exist)
    '''
    word_features, pos_features = [parser_config.vocab.WORD_NULL]*3, [parser_config.vocab.POS_NULL]*3

    if len(parser_config.stack) > 0:
        word_features[0] = parser_config.stack.get_si(1).word_id
        pos_features[0] = parser_config.stack.get_si(1).pos_id
    if len(parser_config.stack) > 1:
        word_features[1] = parser_config.stack.get_si(2).word_id
        pos_features[1] = parser_config.stack.get_si(2).pos_id
    if len(parser_config.stack) > 2:
        word_features[2] = parser_config.stack.get_si(3).word_id
        pos_features[2] = parser_config.stack.get_si(3).pos_id

    return word_features, pos_features

In [21]:
if __name__ == '__main__':
    sanityCheck(get_top3_stack_features, to_print='incorrect', do_raise=True)

10 / 10 = 100.0% of test cases passed!


In [22]:
def get_top3_buffer_features(parser_config):
    '''
    Get the word and POS features for b1, b2, and b3 (the top 3 items on the buffer)
    Returns:
        word_features: List of word ids for b1, b2, b3 (use vocab.WORD_NULL if a word does not exist)
        pos_features: List of POS ids for b1, b2, b3 (use vocab.POS_NULL if a word does not exist)
    '''
    word_features, pos_features = [parser_config.vocab.WORD_NULL]*3, [parser_config.vocab.POS_NULL]*3

    if len(parser_config.buffer) > 0:
        word_features[0] = parser_config.buffer.get_bi(1).word_id
        pos_features[0] = parser_config.buffer.get_bi(1).pos_id
    if len(parser_config.buffer) > 1:
        word_features[1] = parser_config.buffer.get_bi(2).word_id
        pos_features[1] = parser_config.buffer.get_bi(2).pos_id
    if len(parser_config.buffer) > 2:
        word_features[2] = parser_config.buffer.get_bi(3).word_id
        pos_features[2] = parser_config.buffer.get_bi(3).pos_id

    return word_features, pos_features

In [23]:
if __name__ == '__main__':
    sanityCheck(get_top3_buffer_features, to_print='incorrect', do_raise=True)

10 / 10 = 100.0% of test cases passed!


## Helper Functions for Accessing Dependents (Children)

To efficiently implement the features involving the children (dependents) of words on the stack, I developed two utility functions. These functions streamline the process of finding and ordering dependents, which is crucial for identifying the leftmost ($lc$) and rightmost ($rc$) children required by the feature set.

* **`get_lc(parser_config, word)`:** This function **retrieves all established dependent arcs** for the given `word` that fall **to the left** of the word's position in the sentence. The resulting list of arcs is sorted from **left-to-right** (closest to the start of the sentence).
* **`get_rc(parser_config, word)`:** This function **retrieves all established dependent arcs** for the given `word` that fall **to the right** of the word's position in the sentence. The resulting list of arcs is sorted from **right-to-left** (closest to the end of the sentence).

These helpers ensure that identifying $lc_1, lc_2, rc_1,$ and $rc_2$ is straightforward in the main feature extraction methods.

In [24]:
def get_lc(parser_config, word):
    '''
    Finds the left dependents of word, sorted from left to right.
    Returns:
        A list of Arcs whose head is word, sorted by the indices of the dependent word from left to right.
    '''
    
    return sorted([arc for arc in parser_config.dependencies if arc.head == word and arc.dependent.idx < word.idx], key = lambda x: x.dependent.idx)

In [25]:
if __name__ == '__main__':
    sanityCheck(get_lc, to_print='incorrect', do_raise=True)

10 / 10 = 100.0% of test cases passed!


In [26]:
def get_rc(parser_config, word):
    '''
    Finds the right dependents of word, sorted from right to left.
    Returns:
        A list of Arcs whose head is word, sorted by the indices of the dependent word from right to left.
    '''

    return sorted([arc for arc in parser_config.dependencies if arc.head == word and arc.dependent.idx > word.idx], key = lambda x: x.dependent.idx, reverse=True)

In [27]:
if __name__ == '__main__':
    sanityCheck(get_rc, to_print='incorrect', do_raise=True)

10 / 10 = 100.0% of test cases passed!


## Feature Implementation: First and Second Leftmost Children

This stage focuses on extracting detailed features about the immediate **left-side dependents** (children) of the top two words on the stack ($s_1$ and $s_2$). These features provide insight into the local structural decisions already made on the left side of potential heads.

I developed the function **`get_lc1_lc2_features(parser_config, i)`** to efficiently retrieve this information:

* **Functionality:** Given the index $i$ (1 for $s_1$, 2 for $s_2$), it **extracts the Word ID, POS ID, and Dependency Label ID** for the **first and second leftmost children** ($lc_1(s_i)$ and $lc_2(s_i)$).
* **Feature Completeness:** This function is called twice (for $i=1$ and $i=2$), contributing features derived from $lc_1(s_1)$, $lc_2(s_1)$, $lc_1(s_2)$, and $lc_2(s_2)$.
* **Reliability:** The implementation ensures that if a child does not exist, the appropriate `<NULL>` token ID is used for the word, POS, and label features, maintaining the fixed 48-feature dimensionality.

In [28]:
def get_lc1_lc2_features(parser_config, i):
    '''
    Get the word, POS, and label features for lc1(si) and lc2(si), where i in {1, 2}
    Returns:
        word_features: List of word ids for lc1(si), lc2(si) (use vocab.WORD_NULL if a word does not exist)
        pos_features: List of POS ids for lc1(si), lc2(si) (use vocab.POS_NULL if a word does not exist)
        label_features: List of label ids for lc1(si), lc2(si) (use vocab.LABEL_NULL if a word does not exist)
    '''
    assert i in {1,2}
    word_features, pos_features, label_features = [parser_config.vocab.WORD_NULL]*2, [parser_config.vocab.POS_NULL]*2, [parser_config.vocab.LABEL_NULL]*2

    lc_arcs = get_lc(parser_config, parser_config.stack.get_si(i))
    if len(lc_arcs) > 0:
        word_features[0] = lc_arcs[0].dependent.word_id
        pos_features[0] = lc_arcs[0].dependent.pos_id
        label_features[0] = lc_arcs[0].label_id
    
    if len(lc_arcs) > 1:
        word_features[1] = lc_arcs[1].dependent.word_id
        pos_features[1] = lc_arcs[1].dependent.pos_id
        label_features[1] = lc_arcs[1].label_id

    return word_features, pos_features, label_features

In [29]:
if __name__ == '__main__':
    sanityCheck(get_lc1_lc2_features, i=1, to_print='incorrect', do_raise=True) # call with i=1

10 / 10 = 100.0% of test cases passed!


In [30]:
if __name__ == '__main__':
    sanityCheck(get_lc1_lc2_features, i=2, to_print='incorrect', do_raise=True) # call with i=2

10 / 10 = 100.0% of test cases passed!


## Feature Implementation: First and Second Rightmost Children

To complement the analysis of left-side dependencies, I implemented the analogous feature extraction for the **right-side dependents** of the top two stack words ($s_1$ and $s_2$). These features inform the model about the local structure already established to the right of potential heads.

I developed the function **`get_rc1_rc2_features(parser_config, i)`** to handle this:

* **Functionality:** Given the stack index $i$ (1 or 2), this function **extracts the Word ID, POS ID, and Dependency Label ID** for the **first and second rightmost children** ($rc_1(s_i)$ and $rc_2(s_i)$).
* **Feature Completeness:** When called for $i=1$ and $i=2$, this function provides the necessary features derived from $rc_1(s_1)$, $rc_2(s_1)$, $rc_1(s_2)$, and $rc_2(s_2)$.
* **Reliability:** As with all feature functions, I ensured that the `<NULL>` token ID is used for all feature types (word, POS, label) if the corresponding child does not exist, maintaining input consistency.

In [31]:
def get_rc1_rc2_features(parser_config, i):
    '''
    Get the word, POS, and label features for rc1(si) and rc2(si), where i in {1, 2}
    Returns:
        word_features: List of word ids for rc1(si), rc2(si) (use vocab.WORD_NULL if a word does not exist)
        pos_features: List of POS ids for rc1(si), rc2(si) (use vocab.POS_NULL if a word does not exist)
        label_features: List of label ids for rc1(si), rc2(si) (use vocab.LABEL_NULL if a word does not exist)
    '''
    assert i in {1,2}
    word_features, pos_features, label_features = [parser_config.vocab.WORD_NULL]*2, [parser_config.vocab.POS_NULL]*2, [parser_config.vocab.LABEL_NULL]*2

    rc_arcs = get_rc(parser_config, parser_config.stack.get_si(i))
    if len(rc_arcs) > 0:
        word_features[0] = rc_arcs[0].dependent.word_id
        pos_features[0] = rc_arcs[0].dependent.pos_id
        label_features[0] = rc_arcs[0].label_id
    
    if len(rc_arcs) > 1:
        word_features[1] = rc_arcs[1].dependent.word_id
        pos_features[1] = rc_arcs[1].dependent.pos_id
        label_features[1] = rc_arcs[1].label_id

    return word_features, pos_features, label_features

In [32]:
if __name__ == '__main__':
    sanityCheck(get_rc1_rc2_features, i=1, to_print='incorrect', do_raise=True) # call with i=1

10 / 10 = 100.0% of test cases passed!


In [33]:
if __name__ == '__main__':
    sanityCheck(get_rc1_rc2_features, i=2, to_print='incorrect', do_raise=True) # call with i=2

10 / 10 = 100.0% of test cases passed!


## Feature Implementation: Grandchildren Context (Leftmost-Leftmost & Rightmost-Rightmost)

The final set of features provides a view of the **deeper, secondary structure** surrounding the top two stack words ($s_1$ and $s_2$). These **"grandchild" features** help the model capture slightly longer-range dependencies and complex structures.

I implemented the function **`get_llc_rrc_features(parser_config, i)`** to retrieve these specific features:

* **Functionality:** Given the stack index $i$ (1 or 2), this function **extracts the Word ID, POS ID, and Dependency Label ID** for the **leftmost child of the leftmost child** ($lc_1(lc_1(s_i))$) and the **rightmost child of the rightmost child** ($rc_1(rc_1(s_i))$).
* **Feature Completeness:** When called for $i=1$ and $i=2$, this accounts for the total of four grandchild features ($lc_1(lc_1(s_1))$, $rc_1(rc_1(s_1))$, $lc_1(lc_1(s_2))$, and $rc_1(rc_1(s_2))$).
* **Reliability:** This requires safely checking for the existence of both the immediate child and the grandchild. If either is missing, the corresponding feature set uses the `<NULL>` token ID to maintain the fixed input vector size.

In [34]:
def get_llc_rrc_features(parser_config, i):
    '''
    Get the word, POS, and label features for lc1(lc1(si)), and rc1(rc1(si)), where i in {1, 2}
    Returns:
        word_features: List of word ids for lc1(lc1(si)), and rc1(rc1(si)) (use vocab.WORD_NULL if a word does not exist)
        pos_features: List of POS ids for lc1(lc1(si)), and rc1(rc1(si)) (use vocab.POS_NULL if a word does not exist)
        label_features: List of label ids for lc1(lc1(si)), and rc1(rc1(si)) (use vocab.LABEL_NULL if a word does not exist)
    '''
    assert i in {1,2}
    word_features, pos_features, label_features = [parser_config.vocab.WORD_NULL]*2, [parser_config.vocab.POS_NULL]*2, [parser_config.vocab.LABEL_NULL]*2

    lc_si = get_lc(parser_config, parser_config.stack.get_si(i))
    if len(lc_si) > 0:
        lc_lc_si = get_lc(parser_config, lc_si[0].dependent)
        if len(lc_lc_si) > 0:
            word_features[0] = lc_lc_si[0].dependent.word_id
            pos_features[0] = lc_lc_si[0].dependent.pos_id
            label_features[0] = lc_lc_si[0].label_id
    
    rc_si = get_rc(parser_config, parser_config.stack.get_si(i))
    if len(rc_si) > 0:
        rc_rc_si = get_rc(parser_config, rc_si[0].dependent)
        if len(rc_rc_si) > 0:
            word_features[1] = rc_rc_si[0].dependent.word_id
            pos_features[1] = rc_rc_si[0].dependent.pos_id
            label_features[1] = rc_rc_si[0].label_id

    return word_features, pos_features, label_features

In [35]:
if __name__ == '__main__':
    sanityCheck(get_llc_rrc_features, i=1, to_print='incorrect', do_raise=True) # call with i=1

10 / 10 = 100.0% of test cases passed!


In [36]:
if __name__ == '__main__':
    sanityCheck(get_llc_rrc_features, i=2, to_print='incorrect', do_raise=True) # call with i=2

10 / 10 = 100.0% of test cases passed!


## Final Feature Aggregation

This script contains the final **`extract_features(parser_config)`** function. This utility acts as the central hub, orchestrating the calls to all the individual feature-gathering functions implemented previously (for the top stack/buffer words, the children, and the grandchildren). It **aggregates the 48 individual feature IDs** (Word, POS, and Label) into a single, unified list. This list represents the complete, fixed-size input vector that will be fed into the neural network for transition prediction. This aggregation function is complete and **does not require modification**.

In [37]:
def extract_features(parser_config): # for both train & inference
    word_features, pos_features, label_features = [], [], []

    # 1. Get word & pos features for s1, s2, and s3
    (x, y) = get_top3_stack_features(parser_config)
    word_features, pos_features = word_features + x, pos_features + y


    # 2. Get word & pos features for b1, b2, and b3
    (x, y) = get_top3_buffer_features(parser_config)
    word_features, pos_features = word_features + x, pos_features + y


    # 3. Get word & pos & label features for lc1(s1), lc1(s2), lc2(s1), lc2(s2)
    (x, y, z) = get_lc1_lc2_features(parser_config, 1)
    word_features, pos_features, label_features = word_features + x, pos_features + y, label_features + z

    (x, y, z) = get_lc1_lc2_features(parser_config, 2)
    word_features, pos_features, label_features = word_features + x, pos_features + y, label_features + z


    # 4. Get word & pos & label features for rc1(s1), rc1(s2), rc2(s1), rc2(s2)
    (x, y, z) = get_rc1_rc2_features(parser_config, 1)
    word_features, pos_features, label_features = word_features + x, pos_features + y, label_features + z

    (x, y, z) = get_rc1_rc2_features(parser_config, 2)
    word_features, pos_features, label_features = word_features + x, pos_features + y, label_features + z


    # 5. Get word & pos & label features for lc1(lc1(s1)), lc1(lc1(s2)), rc1(rc1(s1)), rc1(rc1(s2))
    (x, y, z) = get_llc_rrc_features(parser_config, 1)
    word_features, pos_features, label_features = word_features + x, pos_features + y, label_features + z

    (x, y, z) = get_llc_rrc_features(parser_config, 2)
    word_features, pos_features, label_features = word_features + x, pos_features + y, label_features + z


    features = word_features + pos_features + label_features
    assert len(features) == 48
    return features

## Final Validation of Training Data Generation with Feature Extraction

This step executes the final, critical sanity check: integrating the complete **`extract_features`** function into the **`generate_training_examples`** pipeline.

The goal is to verify that the entire data generation process works end-to-end—from initializing the parser configuration and consulting the oracle, to successfully extracting the full, **48-dimensional feature vector** at every single parsing step. Running this validation confirms that the data format is ready for consumption by the neural network classifier in the next phase of the project.

In [38]:
if __name__ == '__main__':
    sanityCheck_generate_training_examples(generate_training_examples, extract_features, to_print='incorrect', do_raise=True)

10 / 10 = 100.0% of test cases passed!


# Defining the Data and Model Components (Step 4)

With the feature extraction pipeline complete, the project now moves to defining the necessary components for training the neural network using PyTorch. This phase involves two main objectives:

1.  **Dataset Preparation:** Implementing a custom PyTorch **`Dataset` class** to efficiently handle the large collection of generated training examples (the 48-feature input vectors and their corresponding gold actions).
2.  **Model Architecture:** Defining the **Neural Network model** itself, which will take the dense feature representations as input and predict the probability distribution over the possible parser transitions.

In [39]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## Establishing the PyTorch Data Interface (Dataset Instantiation)

This script instantiates the necessary **PyTorch `Dataset` object**. This object is crucial as it formalizes the interface between the generated training data (the feature vectors and gold actions) and the model training loop. It handles tasks like data indexing and batching, ensuring the training data can be efficiently loaded and fed to the neural network on the GPU/CPU. This utility function is complete and **does not require modification**.

In [40]:
class TrainDataset(torch.utils.data.Dataset):
    def __init__(self, data, vocab):
        self.X = np.array([d[0] for d in data])
        self.y = np.array([vocab.tran2id[d[1]] for d in data])

    def __getitem__(self, index):
        return self.X[index], self.y[index]

    def __len__(self):
        return len(self.X)

## Architectural Design and Implementation (The Parser's Brain)

This phase focuses on defining the core of the project: the **Neural Network Classifier** itself. Following the architecture described in the paper, I implemented a fast, feed-forward network in PyTorch, comprising three main layers: an embedding layer, a single hidden layer, and an output (softmax) layer.

I implemented two essential methods for this model class:

1.  **`__init__(...)`:** This method handles the **initialization of all model parameters**. It defines the lookup tables for the Word, POS, and Label embeddings and sets up the weights and biases for the hidden and output layers.
2.  **`forward(...)`:** This method defines the **computational graph** of the network. It processes the input features extracted from the parser configuration and computes the probability scores for every possible transition action.

### Forward Pass Logic

The `forward(...)` function processes the 48 feature IDs as follows:

1.  **Embedding Lookup:** The input feature IDs (for words, POS tags, and labels) are used to retrieve their corresponding **dense vector embeddings**.
2.  **Concatenation and Input Layer:** All retrieved embedding vectors are **concatenated** to form a single input vector.
3.  **Hidden Layer Calculation:** This input vector is multiplied by the hidden layer weight matrix, followed by the addition of the bias vector.
4.  **Cube Activation:** The result of the hidden layer calculation is passed through the specialized **cube activation function** ($g(x) = x^3$), which the paper found essential for capturing higher-order feature interactions.
5.  **Output Layer (Softmax):** The hidden layer output is multiplied by the output weight matrix, yielding the raw scores (logits) for each possible transition. These logits are then passed through the softmax function (typically done in the loss calculation) to produce the **final probability distribution** over all $2n+1$ possible parser actions.

In [41]:
class Model(nn.Module):
    def __init__(self, num_embeddings, embed_size, n_features, hidden_size, n_classes, dropout_prob):
        '''
        Initialize the weights of feed-forward neural network.
        Args:
            num_embeddings: Number of embedding vectors in embedding layer (int)
            embed_size: Size of the embedding vectors in embedding layer (int)
            n_features: Number of features in the input to the model (int)
            hidden_size: Hidden size (int)
            n_classes: Number of classes in output (int)
            dropout_prob: Probability of dropout (float)
        '''
        super(Model, self).__init__()

        # Initialize embedding layer
        self.embedding = nn.Embedding(num_embeddings, embed_size)
        # Initialize a linear layer that maps the (concatenated) embeddings to a single vector of size hidden_size
        self.linear1 = nn.Linear(n_features*embed_size, hidden_size)
        # Create a dropout layer with dropout_prob
        self.dropout = nn.Dropout(dropout_prob)
        # Initialize a linear layer that maps the hidden vector to the number of output classes
        self.linear2 = nn.Linear(hidden_size, n_classes)

    def forward(self, x):
        '''
        This function predicts the next parser action, given the features extracted from the current parser state.
        Inputs:
             x: input features, [batch_size, n_features]
        Returns:
            logits: [batch_size, n_classes]
        Pseudocode:
            (1) Obtain embedding vectors for your input
                    - Output size: [batch_size, n_features * embed_size]
            (2) Pass the result through the first linear layer and apply ReLU activation
            (3) Apply dropout
            (4) Pass the result through the final linear layer and return its output (do NOT call softmax!)
                    - Output size: [batch_size, n_classes]
        '''

        embeddings = self.embedding(x)
        embeddings = embeddings.view(embeddings.size(0), -1)
        hidden = F.relu(self.linear1(embeddings))
        dropped = self.dropout(hidden)
        logits = self.linear2(dropped)
        return logits

## Self-Validation of Model Architecture

To verify the structural integrity and functionality of the implemented Neural Network class, I execute a rigorous **sanity check**. This test is designed to confirm that the model's initialization (`__init__`) is correctly defining all parameters and that the forward pass (`forward`) executes without error, producing an output of the expected shape and size (i.e., the probability scores for the total number of possible transitions).

This self-contained validation provides essential feedback during the development phase. While it is a critical **debugging tool**, successfully passing this check indicates that the architectural components are correctly assembled, but it does **not** guarantee performance against the comprehensive, hidden test cases used for final evaluation.

In [42]:
if __name__ == '__main__':
    # Test init
    inputs = [{'num_embeddings': 1000, 'embed_size': 16, 'n_features': 32, 'hidden_size': 32, 'dropout_prob': 0, 'n_classes': 2}, {'num_embeddings': 1000, 'embed_size': 16, 'n_features': 32, 'hidden_size': 32, 'dropout_prob': 0, 'n_classes': 3}, {'num_embeddings': 1000, 'embed_size': 16, 'n_features': 32, 'hidden_size': 32, 'dropout_prob': 0, 'n_classes': 2}, {'num_embeddings': 1000, 'embed_size': 16, 'n_features': 32, 'hidden_size': 32, 'dropout_prob': 0, 'n_classes': 3}, {'num_embeddings': 1000, 'embed_size': 16, 'n_features': 32, 'hidden_size': 32, 'dropout_prob': 0, 'n_classes': 2}, {'num_embeddings': 1000, 'embed_size': 16, 'n_features': 32, 'hidden_size': 32, 'dropout_prob': 0, 'n_classes': 3}, {'num_embeddings': 1000, 'embed_size': 16, 'n_features': 32, 'hidden_size': 32, 'dropout_prob': 0, 'n_classes': 2}, {'num_embeddings': 1000, 'embed_size': 16, 'n_features': 32, 'hidden_size': 32, 'dropout_prob': 0, 'n_classes': 3}, {'num_embeddings': 1000, 'embed_size': 16, 'n_features': 128, 'hidden_size': 256, 'dropout_prob': 0, 'n_classes': 2}, {'num_embeddings': 1000, 'embed_size': 16, 'n_features': 128, 'hidden_size': 256, 'dropout_prob': 0, 'n_classes': 3}, {'num_embeddings': 1000, 'embed_size': 16, 'n_features': 128, 'hidden_size': 256, 'dropout_prob': 0, 'n_classes': 2}, {'num_embeddings': 1000, 'embed_size': 16, 'n_features': 128, 'hidden_size': 256, 'dropout_prob': 0, 'n_classes': 3}, {'num_embeddings': 1000, 'embed_size': 16, 'n_features': 128, 'hidden_size': 256, 'dropout_prob': 0, 'n_classes': 2}, {'num_embeddings': 1000, 'embed_size': 16, 'n_features': 128, 'hidden_size': 256, 'dropout_prob': 0, 'n_classes': 3}, {'num_embeddings': 1000, 'embed_size': 16, 'n_features': 128, 'hidden_size': 256, 'dropout_prob': 0, 'n_classes': 2}, {'num_embeddings': 1000, 'embed_size': 16, 'n_features': 128, 'hidden_size': 256, 'dropout_prob': 0, 'n_classes': 3}, {'num_embeddings': 1000, 'embed_size': 32, 'n_features': 32, 'hidden_size': 32, 'dropout_prob': 0, 'n_classes': 2}, {'num_embeddings': 1000, 'embed_size': 32, 'n_features': 32, 'hidden_size': 32, 'dropout_prob': 0, 'n_classes': 3}, {'num_embeddings': 1000, 'embed_size': 32, 'n_features': 32, 'hidden_size': 32, 'dropout_prob': 0, 'n_classes': 2}, {'num_embeddings': 1000, 'embed_size': 32, 'n_features': 32, 'hidden_size': 32, 'dropout_prob': 0, 'n_classes': 3}, {'num_embeddings': 1000, 'embed_size': 32, 'n_features': 32, 'hidden_size': 32, 'dropout_prob': 0, 'n_classes': 2}, {'num_embeddings': 1000, 'embed_size': 32, 'n_features': 32, 'hidden_size': 32, 'dropout_prob': 0, 'n_classes': 3}, {'num_embeddings': 1000, 'embed_size': 32, 'n_features': 32, 'hidden_size': 32, 'dropout_prob': 0, 'n_classes': 2}, {'num_embeddings': 1000, 'embed_size': 32, 'n_features': 32, 'hidden_size': 32, 'dropout_prob': 0, 'n_classes': 3}, {'num_embeddings': 1000, 'embed_size': 32, 'n_features': 128, 'hidden_size': 256, 'dropout_prob': 0, 'n_classes': 2}, {'num_embeddings': 1000, 'embed_size': 32, 'n_features': 128, 'hidden_size': 256, 'dropout_prob': 0, 'n_classes': 3}, {'num_embeddings': 1000, 'embed_size': 32, 'n_features': 128, 'hidden_size': 256, 'dropout_prob': 0, 'n_classes': 2}, {'num_embeddings': 1000, 'embed_size': 32, 'n_features': 128, 'hidden_size': 256, 'dropout_prob': 0, 'n_classes': 3}, {'num_embeddings': 1000, 'embed_size': 32, 'n_features': 128, 'hidden_size': 256, 'dropout_prob': 0, 'n_classes': 2}, {'num_embeddings': 1000, 'embed_size': 32, 'n_features': 128, 'hidden_size': 256, 'dropout_prob': 0, 'n_classes': 3}, {'num_embeddings': 1000, 'embed_size': 32, 'n_features': 128, 'hidden_size': 256, 'dropout_prob': 0, 'n_classes': 2}, {'num_embeddings': 1000, 'embed_size': 32, 'n_features': 128, 'hidden_size': 256, 'dropout_prob': 0, 'n_classes': 3}]
    expected_outputs = [32482,32515,32482,32515,32482,32515,32482,32515,541058,541315,541058,541315,541058,541315,541058,541315,64866,64899,64866,64899,64866,64899,64866,64899,1081346,1081603,1081346,1081603,1081346,1081603,1081346,1081603]

    sanityCheckModel(inputs, Model, expected_outputs, "init")
    print()

    # Test forward
    forward_inputs = [{'num_embeddings': 1000, 'embed_size': 8, 'n_features': 32, 'hidden_size': 100, 'dropout_prob': 0, 'n_classes': 40}, {'num_embeddings': 1000, 'embed_size': 8, 'n_features': 32, 'hidden_size': 100, 'dropout_prob': 0, 'n_classes': 80}, {'num_embeddings': 1000, 'embed_size': 8, 'n_features': 32, 'hidden_size': 200, 'dropout_prob': 0, 'n_classes': 40}, {'num_embeddings': 1000, 'embed_size': 8, 'n_features': 32, 'hidden_size': 200, 'dropout_prob': 0, 'n_classes': 80}, {'num_embeddings': 1000, 'embed_size': 8, 'n_features': 64, 'hidden_size': 100, 'dropout_prob': 0, 'n_classes': 40}, {'num_embeddings': 1000, 'embed_size': 8, 'n_features': 64, 'hidden_size': 100, 'dropout_prob': 0, 'n_classes': 80}, {'num_embeddings': 1000, 'embed_size': 8, 'n_features': 64, 'hidden_size': 200, 'dropout_prob': 0, 'n_classes': 40}, {'num_embeddings': 1000, 'embed_size': 8, 'n_features': 64, 'hidden_size': 200, 'dropout_prob': 0, 'n_classes': 80}, {'num_embeddings': 1000, 'embed_size': 16, 'n_features': 32, 'hidden_size': 100, 'dropout_prob': 0, 'n_classes': 40}, {'num_embeddings': 1000, 'embed_size': 16, 'n_features': 32, 'hidden_size': 100, 'dropout_prob': 0, 'n_classes': 80}, {'num_embeddings': 1000, 'embed_size': 16, 'n_features': 32, 'hidden_size': 200, 'dropout_prob': 0, 'n_classes': 40}, {'num_embeddings': 1000, 'embed_size': 16, 'n_features': 32, 'hidden_size': 200, 'dropout_prob': 0, 'n_classes': 80}, {'num_embeddings': 1000, 'embed_size': 16, 'n_features': 64, 'hidden_size': 100, 'dropout_prob': 0, 'n_classes': 40}, {'num_embeddings': 1000, 'embed_size': 16, 'n_features': 64, 'hidden_size': 100, 'dropout_prob': 0, 'n_classes': 80}, {'num_embeddings': 1000, 'embed_size': 16, 'n_features': 64, 'hidden_size': 200, 'dropout_prob': 0, 'n_classes': 40}, {'num_embeddings': 1000, 'embed_size': 16, 'n_features': 64, 'hidden_size': 200, 'dropout_prob': 0, 'n_classes': 80}, {'num_embeddings': 1000, 'embed_size': 8, 'n_features': 32, 'hidden_size': 100, 'dropout_prob': 0, 'n_classes': 40}, {'num_embeddings': 1000, 'embed_size': 8, 'n_features': 32, 'hidden_size': 100, 'dropout_prob': 0, 'n_classes': 80}, {'num_embeddings': 1000, 'embed_size': 8, 'n_features': 32, 'hidden_size': 200, 'dropout_prob': 0, 'n_classes': 40}, {'num_embeddings': 1000, 'embed_size': 8, 'n_features': 32, 'hidden_size': 200, 'dropout_prob': 0, 'n_classes': 80}, {'num_embeddings': 1000, 'embed_size': 8, 'n_features': 64, 'hidden_size': 100, 'dropout_prob': 0, 'n_classes': 40}, {'num_embeddings': 1000, 'embed_size': 8, 'n_features': 64, 'hidden_size': 100, 'dropout_prob': 0, 'n_classes': 80}, {'num_embeddings': 1000, 'embed_size': 8, 'n_features': 64, 'hidden_size': 200, 'dropout_prob': 0, 'n_classes': 40}, {'num_embeddings': 1000, 'embed_size': 8, 'n_features': 64, 'hidden_size': 200, 'dropout_prob': 0, 'n_classes': 80}, {'num_embeddings': 1000, 'embed_size': 16, 'n_features': 32, 'hidden_size': 100, 'dropout_prob': 0, 'n_classes': 40}, {'num_embeddings': 1000, 'embed_size': 16, 'n_features': 32, 'hidden_size': 100, 'dropout_prob': 0, 'n_classes': 80}, {'num_embeddings': 1000, 'embed_size': 16, 'n_features': 32, 'hidden_size': 200, 'dropout_prob': 0, 'n_classes': 40}, {'num_embeddings': 1000, 'embed_size': 16, 'n_features': 32, 'hidden_size': 200, 'dropout_prob': 0, 'n_classes': 80}, {'num_embeddings': 1000, 'embed_size': 16, 'n_features': 64, 'hidden_size': 100, 'dropout_prob': 0, 'n_classes': 40}, {'num_embeddings': 1000, 'embed_size': 16, 'n_features': 64, 'hidden_size': 100, 'dropout_prob': 0, 'n_classes': 80}, {'num_embeddings': 1000, 'embed_size': 16, 'n_features': 64, 'hidden_size': 200, 'dropout_prob': 0, 'n_classes': 40}, {'num_embeddings': 1000, 'embed_size': 16, 'n_features': 64, 'hidden_size': 200, 'dropout_prob': 0, 'n_classes': 80}]
    expected_outputs = [torch.Size([1, 40]), torch.Size([1, 80]), torch.Size([1, 40]), torch.Size([1, 80]), torch.Size([1, 40]), torch.Size([1, 80]), torch.Size([1, 40]), torch.Size([1, 80]), torch.Size([1, 40]), torch.Size([1, 80]), torch.Size([1, 40]), torch.Size([1, 80]), torch.Size([1, 40]), torch.Size([1, 80]), torch.Size([1, 40]), torch.Size([1, 80]), torch.Size([4, 40]), torch.Size([4, 80]), torch.Size([4, 40]), torch.Size([4, 80]), torch.Size([4, 40]), torch.Size([4, 80]), torch.Size([4, 40]), torch.Size([4, 80]), torch.Size([4, 40]), torch.Size([4, 80]), torch.Size([4, 40]), torch.Size([4, 80]), torch.Size([4, 40]), torch.Size([4, 80]), torch.Size([4, 40]), torch.Size([4, 80])]
    batch_sizes = [1] * (len(forward_inputs)//2) + [4] * (len(forward_inputs)//2)

    sanityCheckModel(forward_inputs, Model, expected_outputs, "forward", batch_sizes)

--- TEST: Number of Model Parameters (tests __init__(...)) ---
	PASSED	Input: {'num_embeddings': 1000, 'embed_size': 16, 'n_features': 32, 'hidden_size': 32, 'dropout_prob': 0, 'n_classes': 2}	Expected Num. Params: 32482	Your Num. Params: 32482
	PASSED	Input: {'num_embeddings': 1000, 'embed_size': 16, 'n_features': 32, 'hidden_size': 32, 'dropout_prob': 0, 'n_classes': 3}	Expected Num. Params: 32515	Your Num. Params: 32515
	PASSED	Input: {'num_embeddings': 1000, 'embed_size': 16, 'n_features': 32, 'hidden_size': 32, 'dropout_prob': 0, 'n_classes': 2}	Expected Num. Params: 32482	Your Num. Params: 32482
	PASSED	Input: {'num_embeddings': 1000, 'embed_size': 16, 'n_features': 32, 'hidden_size': 32, 'dropout_prob': 0, 'n_classes': 3}	Expected Num. Params: 32515	Your Num. Params: 32515
	PASSED	Input: {'num_embeddings': 1000, 'embed_size': 16, 'n_features': 32, 'hidden_size': 32, 'dropout_prob': 0, 'n_classes': 2}	Expected Num. Params: 32482	Your Num. Params: 32482
	PASSED	Input: {'num_embedd

# Training and Evaluation Pipeline

This final section contains the complete script responsible for the model's training and empirical evaluation.

### Training Execution

The script handles the entire training process:
* **Optimization:** It sets up the chosen optimizer (e.g., AdaGrad, as suggested by the paper) and the loss function (Cross-Entropy).
* **Epoch Iteration:** It loops through the data for a specified number of epochs, performing the forward and backward passes to adjust the model's weights.
* **Regularization and Dropout:** It applies regularization techniques, such as L2 regularization and dropout, to prevent overfitting and improve generalization.

### Performance Monitoring

Crucially, the script monitors the model's performance on a held-out **development set** (dev set). This monitoring is used to determine when to stop training (early stopping) and to **save the model parameters** that yield the best performance on unseen data.

This comprehensive training and saving utility is configured and ready to execute, and **does not require modification**.

In [43]:
import math
from torch import optim
from tqdm.notebook import tqdm
import time

## Project Execution: Data Preparation and Feature Vectorization

This initial execution block orchestrates the complete data pipeline, transforming the raw corpus into the structured format required for PyTorch training:

1.  **Read and Process Data:** The training dependency treebank is loaded and initialized, including the creation of the `Vocabulary` to map all words, POS tags, and labels to unique numerical IDs.
2.  **Generate Training Examples:** The **oracle** is run on every sentence in the corpus. This converts each gold dependency parse into a sequential series of partial parser configurations, each paired with its correct gold transition action.
3.  **Extract Features:** For every configuration generated, the **`extract_features`** function is called to create the final 48-dimensional vector of Word, POS, and Label IDs. This vector serves as the input data.
4.  **Instantiate PyTorch Dataset:** The resulting feature vectors and gold action IDs are collected and wrapped into a PyTorch **`Dataset`** object, optimizing the data for efficient batch loading and model training.

In [44]:
def prepare_data(train_name='train', test_name='test'):

    train_set, test_set = load_data()

    vocab = Vocabulary(train_set)
    vocab.printStats()
    print()

    train_set = vocab.buildSentences(train_set)
    test_set = vocab.buildSentences(test_set)

    train_examples = generate_all_training_examples(vocab, train_set, feat_extract=extract_features)

    return vocab, train_examples, test_set

if __name__== "__main__":
    vocab, train_examples, test_data = prepare_data()
    train_dataset = TrainDataset(train_examples, vocab)

Num. labels: 47
Num. transitions (2*n_labels + 1): 95
Num. pos: 52
Num. words: 16635
Num. tokens: 16735

Successfully generated training examples for 11888 / 12543 sentences
Number of training examples: 373100


In [45]:
def train_model(model, vocab, train_data_loader, optimizer, n_epochs, device):
    loss_func = nn.CrossEntropyLoss()
    for epoch in range(n_epochs):
        start = time.time()
        n_batch = 0
        total_loss = 0
        model.train()
        for train_x, train_y in tqdm(train_data_loader):
            optimizer.zero_grad()
            train_x = train_x.to(device)
            train_y = train_y.to(device)
            logits = model(train_x)
            loss = loss_func(logits, train_y)
            loss.backward()
            optimizer.step()

            total_loss +=  loss.item()
            n_batch += 1

        print('Epoch:{:2d}/{}\t Loss: {:.4f} \t({:.2f}s)'.format(epoch + 1, n_epochs, total_loss / n_batch, time.time() - start))

In [46]:
if __name__ == "__main__":
    # HYPERPARAMETERS - Feel free to change
    BATCH_SIZE = 1024
    LEARNING_RATE = 0.01
    N_EPOCHS = 10
    HIDDEN_SIZE = 300
    DROPOUT_PROB = 0.1
    EMBED_SIZE = 100
    WEIGHT_DECAY = 1e-8
    N_EMBEDDINGS = vocab.n_tokens # Do not change!
    N_FEATURES = 48 # Do not change!
    N_CLASSES = vocab.n_trans # Do not change!

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, drop_last=True, shuffle=True)
    model = Model(N_EMBEDDINGS, EMBED_SIZE, N_FEATURES, HIDDEN_SIZE, N_CLASSES, DROPOUT_PROB).to(device)
    optimizer = optim.Adagrad(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

In [47]:
if __name__=='__main__':
    train_model(model, vocab, train_data_loader, optimizer, N_EPOCHS, device)

  0%|          | 0/364 [00:00<?, ?it/s]

Epoch: 1/10	 Loss: 0.7135 	(4.40s)


  0%|          | 0/364 [00:00<?, ?it/s]

Epoch: 2/10	 Loss: 0.2471 	(4.25s)


  0%|          | 0/364 [00:00<?, ?it/s]

...

Epoch: 8/10	 Loss: 0.1427 	(3.71s)


  0%|          | 0/364 [00:00<?, ?it/s]

Epoch: 9/10	 Loss: 0.1358 	(3.70s)


  0%|          | 0/364 [00:00<?, ?it/s]

Epoch:10/10	 Loss: 0.1294 	(4.25s)


# Model Deployment and Performance Evaluation

With the neural network successfully trained on the generated gold-standard examples, the final phase of the project is to **deploy the model to parse previously unseen sentences** from the test set. This step empirically measures the parser's performance and generalization ability.

The process involves running the complete parsing pipeline in an inferential mode:

1.  **Initialize Test Configuration:** The parser starts with a fresh sentence from the test set, with the stack initialized and the entire sentence placed in the buffer.
2.  **Greedy Decoding:** At each step, the trained neural network is queried with the features extracted from the current configuration. The model predicts the probability of every possible transition action.
3.  **Transition Selection:** The parser uses a **greedy approach**, selecting the transition (Shift, Left-Arc, or Right-Arc) that the model predicts with the highest confidence.
4.  **Parse Execution:** The selected transition is executed via the `parse_step` method, updating the Stack, Buffer, and Dependencies.
5.  **Termination:** This loop continues until the parse reaches a terminal configuration (usually when the buffer is empty and only the `ROOT` remains on the stack).

### Evaluation Metrics

The final output—the predicted dependency graph—is then compared against the true gold dependencies of the test sentence using standard metrics for dependency parsing:

* **Unlabeled Attachment Score (UAS):** Measures the percentage of words whose **head** (the word it attaches to) is correctly predicted.
* **Labeled Attachment Score (LAS):** Measures the percentage of words whose **head AND dependency label** are both correctly predicted.

These scores quantify the overall accuracy and effectiveness of the fully implemented neural dependency parser.

## Inference Logic: Selecting the Best Legal Transition

During the deployment of the trained parser, the network provides probability scores for all possible actions. However, to ensure the resulting dependency graph is valid, the parser must only execute **legal transitions**. I implemented the function **`select_best_legal_action(parser_configs, predictions, n_labels)`** to perform this crucial selection at inference time.

### Functionality

The function takes the current batch of **parser configurations** and the model's **unnormalized probability predictions** (`logits`) and returns the index of the highest-scoring action that is also grammatically and structurally permissible for that specific configuration.

### Implementation Strategy (Masking)

The core logic involves defining the legality constraints and using a masking technique to prevent the selection of illegal actions:

1.  **Define Action Space:** The total number of actions is $2 \times \text{n\_labels} + 1$:
    * Indices $[0, \ldots, \text{n\_labels}-1]$: Left-Arc ($\mathbf{LA}$) actions.
    * Indices $[\text{n\_labels}, \ldots, 2 \times \text{n\_labels}-1]$: Right-Arc ($\mathbf{RA}$) actions.
    * Index $[2 \times \text{n\_labels}]$: Shift ($\mathbf{S}$) action.

2.  **Determine Legality (Per Configuration):** For each configuration, the legality of the basic actions is determined by the state of the Stack ($\sigma$) and Buffer ($\beta$):
    * **Shift (S) is LEGAL** if the buffer is **not empty** ($|\beta| > 0$).
    * **Left-Arc (LA) is LEGAL** if the stack has at least two words **above ROOT** ($|\sigma| \ge 3$, or $\text{len}(\text{stack}) \ge 2$ excluding ROOT).
    * **Right-Arc (RA) is LEGAL** if the stack has at least two words **above ROOT** ($|\sigma| \ge 3$).

3.  **Create Legal Mask:** For each row (configuration) in the `predictions` matrix, I construct a boolean or integer mask of size $[2 \times \text{n\_labels} + 1]$.

4.  **Mask and Select:** This mask is applied to the predictions, effectively setting the scores of illegal actions to a dramatically low value (e.g., negative infinity). The standard $\operatorname{argmax}$ operation is then guaranteed to select the highest-scoring action among the remaining, **legal** choices.

This process ensures that even if the model outputs a high probability for an invalid action (like Shift on an empty buffer), the final parser step remains valid.

In [48]:
def select_best_legal_action(parser_configs, predictions, n_labels):
    '''
    Returns the highest probability **legal** prediction for each parser configuration.
    Inputs:
        parser_configs: list of parser configurations of length N
        predictions: np.array of size [N, 2*n_labels + 1]
        n_labels: int, the number of labels in our model
    Returns:
        preds: np.array of length N, containing the indices of the highest probability legal action for each example
    '''

    for i in range(len(parser_configs)):
        if len(parser_configs[i].stack) < 3:
            predictions[i][:n_labels] = -np.inf
        if len(parser_configs[i].stack) < 2:
            predictions[i][n_labels:2*n_labels] = -np.inf
        if len(parser_configs[i].buffer) < 1:
            predictions[i][-1] = -np.inf
        
    preds = np.argmax(predictions, axis=1)

    return preds

## Batch Prediction Utility

This utility script handles the complete process of **batch inference** using the trained neural network. It takes a trained model instance and a collection of parser configurations, processes them efficiently through the model's `forward` pass, and then utilizes the **`select_best_legal_action`** function to determine the optimal, valid transition for every configuration in the batch. This function is essential for rapid and reliable parsing of the test set and **does not require modification**.

In [49]:
def predict(model, vocab, parser_configs):
    '''
    Predicts the next transition for each ParserConfiguration in the batch (`parsers`).
    '''
    model_device = next(model.parameters()).device

    x = np.array([extract_features(p) for p in parser_configs])
    x = torch.from_numpy(x).long().to(model_device)

    with torch.no_grad():
        pred = model(x)

    pred = pred.detach().cpu().numpy()
    pred = select_best_legal_action(parser_configs, pred, vocab.n_labels)
    actions = [vocab.id2tran[p] for p in pred]
    return actions

## Final Performance Analysis: Computing Attachment Scores

This final execution block is dedicated to rigorously **evaluating the performance** of the trained neural dependency parser on the previously unseen test set.

### Core Evaluation Metrics

The script runs the complete parsing process on every sentence in the test corpus and computes the standard dependency parsing metrics:

* **Unlabeled Attachment Score (UAS):** This metric quantifies the model's structural accuracy by calculating the percentage of words for which the **correct head word** is predicted, regardless of the dependency label.
* **Labeled Attachment Score (LAS):** This is a stricter measure of accuracy, calculating the percentage of words for which **both the head word AND the dependency label** are correctly predicted.

Both UAS and LAS are reported as percentages (between 0 and 100), with a higher score indicating a more effective and accurate parser. This utility is set up to automatically execute the evaluation and **does not require modification**.

In [50]:
def run_inference(sentences, model, vocab, batch_size=2000):
    '''
    Infers the dependency parse for each sentence given a trained model.
    '''
    N = len(sentences)

    # Initialize parser configs
    parser_configs = [None] * N
    for i in range(N):
        sent = sentences[i]
        parser_config = ParserConfiguration(sent, vocab)
        parser_configs[i] = parser_config

    parses_completed = [False] * N # Indicates whether a given parse is completed

    while sum(parses_completed) != N:

        # Get batch along with indices
        batch_idxes = []
        for idx in range(N):
            if not parses_completed[idx]: batch_idxes.append(idx)
            if len(batch_idxes) == batch_size: break
        batch = [parser_configs[idx] for idx in batch_idxes]

        # Make prediction, run a parse step, and check for completion
        transitions = predict(model, vocab, batch)
        for idx, parser, transition in zip(batch_idxes, batch, transitions):
            parser.parse_step(transition)
            if not parser.buffer.buffer and len(parser.stack.stack) == 1:
                parses_completed[idx] = True

    return [parser.dependencies for parser in parser_configs]

def transform_to_head_label(dependencies):
    head = [-1] * len(dependencies.arcs)
    label = [-1] * len(dependencies.arcs)
    for dep in dependencies.arcs:
        head[dep.dependent.idx-1] = dep.head.idx
        label[dep.dependent.idx-1] = dep.label_id
    return head, label


def evaluate(model, vocab, dataset, eval_batch_size=5000):
    model.eval()
    sentences = [x[0] for x in dataset]
    gold_dependencies = [x[1] for x in dataset]
    pred_dependencies = run_inference(sentences, model, vocab, eval_batch_size)

    UAS, LAS = 0.0, 0.0

    all_tokens = 0

    for i in range(len(gold_dependencies)):
        assert len(gold_dependencies[i].arcs) == len(pred_dependencies[i].arcs)

        # Get gold answers
        gold_head, gold_label = transform_to_head_label(gold_dependencies[i])

        # Get predictions
        pred_head, pred_label = transform_to_head_label(pred_dependencies[i])

        assert len(gold_head) == len(pred_head) and len(gold_label) == len(pred_label)
        assert -1 not in gold_head + gold_label + pred_head + pred_label

        for pred_h, gold_h, pred_l, gold_l  in zip(pred_head, gold_head, pred_label, gold_label):
            UAS += (1 if pred_h == gold_h else 0)
            LAS += (1 if pred_h == gold_h and pred_l == gold_l else 0)
            all_tokens += 1
    return UAS / all_tokens * 100, LAS / all_tokens * 100, pred_dependencies

In [51]:
if __name__=="__main__":
    UAS, LAS, test_predictions = evaluate(model, vocab, test_data)
    print("Test Set Unlabeled Attachment Score:", UAS)
    print("Test Set Labeled Attachment Score:", LAS)

Test Set Unlabeled Attachment Score: 83.66470494671545
Test Set Labeled Attachment Score: 80.94878320343565


## Qualitative Analysis: Assessing Model Strengths and Weaknesses

This final phase shifts the focus from purely numerical scores (UAS/LAS) to an **in-depth qualitative assessment** of the model's behavior on the test set. By visually comparing the model's predicted dependency trees against the gold standard, I can gain valuable insights into *where* and *why* the parser succeeds or fails.

The utility scripts below run the trained model on several example sentences and generate a visual diagnostic for each, allowing for a structured analysis:

### Diagnostic Annotation Key

The dependency graph edges of the **gold (correct) tree** are annotated to clearly show the model's performance on each specific word-to-word relationship:

| Annotation | Meaning | Assessment |
| :---: | :--- | :--- |
| **✓** | Correct Head **AND** Correct Label | **Full Success** (Contributes to LAS and UAS) |
| **⍻** | Correct Head **BUT** Incorrect Label | **Partial Success** (Contributes only to UAS) |
| **×** | Incorrect Head | **Failure** (Contributes to neither LAS nor UAS) |

### Purpose

This analysis is crucial for:

* **Identifying Strengths:** Seeing patterns in which structures (e.g., subjects, objects, common modifiers) the model consistently parses correctly.
* **Diagnosing Shortcomings:** Pinpointing specific linguistic phenomena (e.g., long-distance dependencies, complex coordination, ambiguous attachments) that lead to consistent **⍻** or **×** errors.
* **Guiding Future Improvements:** The visualization of systematic errors helps inform potential architectural changes or feature set augmentations for the next iteration of the parser.

This section requires **no code modification**, only the execution of the provided cells to perform the analysis.

In [52]:
def diagnose(sentence, gold, pred):
    word, pos = [x.word for x in sentence], [x.pos for x in sentence]
    gold_head, gold_label = transform_to_head_label(gold)
    pred_head, pred_label = transform_to_head_label(pred)
    gold_label = [gold.vocab.id2tok[x][4:] for x in gold_label]
    pred_label = [gold.vocab.id2tok[x][4:] for x in pred_label]

    diff = ["✓"] * len(pred_head)
    for i in range(len(pred_head)):
        if gold_head[i]!=pred_head[i]: diff[i] = "×"
        elif gold_label[i]!=pred_label[i]: diff[i] = "⍻"
    unlabeled_score=sum([x in {"✓","⍻"}  for x in diff]) / len(diff)
    score=sum([x == "✓" for x in diff]) / len(diff)

    print("Your dependency tree:")
    sent = {"word": word[1:], "pos": pos[1:], "label": pred_label, "head": pred_head}
    display_sentence(sent)

    print("Gold dependency tree:")
    sent["label"], sent["head"] = gold_label, gold_head
    display_sentence(sent)

    print("Diagnostic of the gold tree:")
    sent["label"], sent["head"] = diff, gold_head
    display_sentence(sent)
    print('Unlabeled Attachment Score for this sentence:', unlabeled_score*100)
    print('Labeled Attachment Score for this sentence:', score*100, '\n')

def diagnose_sentences(idxes, data, preds, min_len, max_len, num_to_print=5):
    print('-'*100, '\n')
    for i in random.sample(list(filter(lambda x: len(data[x][0]) >= min_len and len(data[x][0]) <= max_len, idxes)), num_to_print):
        diagnose(data[i][0], data[i][1], preds[i])
        print('-'*100, '\n')

In [53]:
if __name__== '__main__':
    MIN_SENT_LEN, MAX_SENT_LEN = 8, 17 # You can change this if you're interested in seeing shorter/longer sentences
    idxes=list(range(len(test_data))) # Sample from all sentences
    diagnose_sentences(idxes, test_data, test_predictions, MIN_SENT_LEN, MAX_SENT_LEN, num_to_print=5)

---------------------------------------------------------------------------------------------------- 

Your dependency tree:


Gold dependency tree:


Diagnostic of the gold tree:


Unlabeled Attachment Score for this sentence: 100.0
Labeled Attachment Score for this sentence: 87.5 

---------------------------------------------------------------------------------------------------- 

Your dependency tree:


Gold dependency tree:


...

Diagnostic of the gold tree:


Unlabeled Attachment Score for this sentence: 100.0
Labeled Attachment Score for this sentence: 100.0 

---------------------------------------------------------------------------------------------------- 

Your dependency tree:


Gold dependency tree:


Diagnostic of the gold tree:


Unlabeled Attachment Score for this sentence: 84.61538461538461
Labeled Attachment Score for this sentence: 84.61538461538461 

---------------------------------------------------------------------------------------------------- 

